In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
%matplotlib inline
import os
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/Nikhil_Chamle/Desktop/Final_Projects/2. Debt collection/DataFile1.csv')

In [3]:
df = data.copy()

In [4]:
pd.set_option('display.max_columns', 50)

In [5]:
df

,EntityID,OriginalCreditor[Redacted],AccountID,CurrentBalance,DebtLoadPrincipal,BalanceAtDebtLoad,PurchasePrice,ProductOrDebtType,CollectionStatus,IsStatBarred,ClosureReason,InBankruptcy,AccountInsolvencyType,CustomerInsolvencyType,IsLegal,LastPaymentAmount,LastPaymentMethod,NumLiableParties,CustomerAge,NumPhones,NumEmails,NumAddresses,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,932,Creditor 1,3677,0.00,"1,160.20","1,160.20",4.22,Other,PAID_IN_FULL,N,NaN,N,NaN,NaN,Y,10.00,Cheque,1.0,53.0,0,0,1,NaN,NaN,NaN,NaN
1,160,Creditor 2,4276,182.90,182.90,182.90,4.22,Other,CANCELLED_WITHDRAWN,Y,NaN,N,NaN,NaN,N,NaN,NaN,1.0,NaN,0,0,1,NaN,NaN,NaN,NaN
2,932,Creditor 1,8525,0.00,538.57,538.57,4.22,Other,PAID_IN_FULL,N,NaN,N,NaN,NaN,N,5.37,Cheque,1.0,NaN,1,0,1,NaN,NaN,NaN,NaN
3,160,Creditor 2,9859,"8,279.50","8,279.50","8,279.50",4.22,Other,PASSIVE,Y,NaN,N,NaN,NaN,N,NaN,NaN,1.0,NaN,1,0,1,NaN,NaN,NaN,NaN
4,932,Creditor 1,12807,0.00,523.00,523.00,4.22,Other,PAID_IN_FULL,Y,NaN,N,NaN,NaN,Y,5.00,Cheque,1.0,46.0,2,0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406418,99901565,Creditor 50,990495842,448.20,448.20,448.20,7.38,Finance Company - Other,ACTIVE,N,NaN,N,NaN,NaN,N,NaN,NaN,1.0,36.0,1,1,1,NaN,NaN,NaN,NaN
406419,99901565,Creditor 50,990495843,"1,678.37","1,678.37","1,678.37",7.38,Finance Company - Other,ACTIVE,N,NaN,N,NaN,NaN,N,NaN,NaN,1.0,37.0,0,1,1,NaN,NaN,NaN,NaN
406420,99901565,Creditor 50,990495844,"3,512.60","3,512.60","3,512.60",7.38,Finance Company - Other,ACTIVE,N,NaN,N,NaN,NaN,N,NaN,NaN,1.0,71.0,1,1,1,NaN,NaN,NaN,NaN
406421,99901565,Creditor 50,990495845,"4,477.31","4,477.31","4,477.31",7.38,Finance Company - Other,ACTIVE,N,NaN,N,NaN,NaN,N,NaN,NaN,1.0,38.0,1,1,1,NaN,NaN,NaN,NaN


In [6]:
# Dropping columns

df = df.drop(['Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'], axis = 1 )

In [7]:
# checking null values

df.isnull().sum()/len(df) * 100

EntityID                       0.000000
OriginalCreditor[Redacted]     0.000000
AccountID                      0.000000
CurrentBalance                 0.000000
DebtLoadPrincipal              0.000000
BalanceAtDebtLoad              0.000000
PurchasePrice                  0.662364
ProductOrDebtType              0.000000
CollectionStatus               0.000000
IsStatBarred                   0.000000
ClosureReason                 97.778177
InBankruptcy                   0.000000
AccountInsolvencyType         99.929876
CustomerInsolvencyType        97.900955
IsLegal                        0.000000
LastPaymentAmount             74.416556
LastPaymentMethod             74.416556
NumLiableParties               0.030018
CustomerAge                    7.254019
NumPhones                      0.000000
NumEmails                      0.000000
NumAddresses                   0.000000
dtype: float64

In [8]:
# checking duplicates

df.duplicated().sum()

0

In [9]:
#dropping columns with max null values

df = df.drop(['ClosureReason', 'AccountInsolvencyType', 'CustomerInsolvencyType', 'LastPaymentMethod', 'LastPaymentAmount'], axis = 1 )

In [10]:
# Converting columns to float
columns_to_convert = ['CurrentBalance', 'DebtLoadPrincipal', 'BalanceAtDebtLoad']
 
for column in columns_to_convert:
    df[column] = df[column].str.replace(',', '').astype(float)

In [11]:
# converting negative age to positive

df['CustomerAge'] = df['CustomerAge'].abs()

In [12]:
# imputing null values in age column

mean_age = df['CustomerAge'].mean()
df['CustomerAge'].fillna(mean_age, inplace=True)

In [13]:
# dropping null rows

df.dropna(inplace=True)

In [14]:
# dropping insignificant columns for our analysis

df = df.drop(['EntityID', 'AccountID', 'OriginalCreditor[Redacted]'], axis = 1 )

In [15]:
# renaming column

df.rename(columns={'CurrentBalance': 'Debt_Pending'}, inplace=True)

In [16]:
df

,Debt_Pending,DebtLoadPrincipal,BalanceAtDebtLoad,PurchasePrice,ProductOrDebtType,CollectionStatus,IsStatBarred,InBankruptcy,IsLegal,NumLiableParties,CustomerAge,NumPhones,NumEmails,NumAddresses
0,0.00,1160.20,1160.20,4.22,Other,PAID_IN_FULL,N,N,Y,1.0,53.000000,0,0,1
1,182.90,182.90,182.90,4.22,Other,CANCELLED_WITHDRAWN,Y,N,N,1.0,45.730273,0,0,1
2,0.00,538.57,538.57,4.22,Other,PAID_IN_FULL,N,N,N,1.0,45.730273,1,0,1
3,8279.50,8279.50,8279.50,4.22,Other,PASSIVE,Y,N,N,1.0,45.730273,1,0,1
4,0.00,523.00,523.00,4.22,Other,PAID_IN_FULL,Y,N,Y,1.0,46.000000,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406418,448.20,448.20,448.20,7.38,Finance Company - Other,ACTIVE,N,N,N,1.0,36.000000,1,1,1
406419,1678.37,1678.37,1678.37,7.38,Finance Company - Other,ACTIVE,N,N,N,1.0,37.000000,0,1,1
406420,3512.60,3512.60,3512.60,7.38,Finance Company - Other,ACTIVE,N,N,N,1.0,71.000000,1,1,1
406421,4477.31,4477.31,4477.31,7.38,Finance Company - Other,ACTIVE,N,N,N,1.0,38.000000,1,1,1


In [17]:
#dropping columns

df = df.drop(['Debt_Pending', 'DebtLoadPrincipal', 'InBankruptcy', 'IsLegal'], axis = 1 )

In [18]:
# mapping values for statue barred column

df['IsStatBarred'] = df['IsStatBarred'].map({'Y': 1, 'N': 0})

In [19]:
df.columns

Index(['BalanceAtDebtLoad', 'PurchasePrice', 'ProductOrDebtType',
       'CollectionStatus', 'IsStatBarred', 'NumLiableParties', 'CustomerAge',
       'NumPhones', 'NumEmails', 'NumAddresses'],
      dtype='object')

In [20]:
df

,BalanceAtDebtLoad,PurchasePrice,ProductOrDebtType,CollectionStatus,IsStatBarred,NumLiableParties,CustomerAge,NumPhones,NumEmails,NumAddresses
0,1160.20,4.22,Other,PAID_IN_FULL,0,1.0,53.000000,0,0,1
1,182.90,4.22,Other,CANCELLED_WITHDRAWN,1,1.0,45.730273,0,0,1
2,538.57,4.22,Other,PAID_IN_FULL,0,1.0,45.730273,1,0,1
3,8279.50,4.22,Other,PASSIVE,1,1.0,45.730273,1,0,1
4,523.00,4.22,Other,PAID_IN_FULL,1,1.0,46.000000,2,0,1
...,...,...,...,...,...,...,...,...,...,...
406418,448.20,7.38,Finance Company - Other,ACTIVE,0,1.0,36.000000,1,1,1
406419,1678.37,7.38,Finance Company - Other,ACTIVE,0,1.0,37.000000,0,1,1
406420,3512.60,7.38,Finance Company - Other,ACTIVE,0,1.0,71.000000,1,1,1
406421,4477.31,7.38,Finance Company - Other,ACTIVE,0,1.0,38.000000,1,1,1


In [21]:
categorical_features = ['ProductOrDebtType', 'CollectionStatus']

for feature in categorical_features:
    df[feature] = df[feature].astype('category')

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
y = df['IsStatBarred'].astype(int)  # Target variable
X = df.drop(columns=['IsStatBarred'])  # Features

In [24]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Creating a CatBoost Pool object with categorical features specified
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)

# Initializing the CatBoost model
catboost_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0)

# Fitting the CatBoost model
catboost_model.fit(train_pool)

# Making predictions on the test set
y_pred = catboost_model.predict(X_test)

# Calculating evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Printing the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"ROC AUC: {roc_auc}")


Accuracy: 0.9613493056501864
Precision: 0.9686638211167905
Recall: 0.976346865428536
F1-Score: 0.9724901687622339
ROC AUC: 0.9513747528462811
